# Mode Selection
Set finetuning = 1 if you want to finetune, or set finetuning = 0 if you want to just load the saved model to get inference.

In [1]:
finetuning = 1
load_and_run = not finetuning

# Load Data

In [2]:
import pandas as pd

# data = pd.read_csv('/kaggle/input/gb-all-necessary/train_test_toxicity_data.csv')
data = pd.read_csv('train_test_hatespeech_data.csv')
data

c:\Users\joyau\anaconda3\envs\pytorch_gpu\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


FileNotFoundError: [Errno 2] No such file or directory: 'train_test_hatespeech_data.csv'

# Train, Test, Validation Split
We are going to consider the data as test data if the prediction between original sentence and NER converted sentence become unequal.

In [3]:
temp = data[data['test']!= 1]
test = data[data['test']== 1]
validation = temp.groupby('_original_label', group_keys=False).apply(lambda x: x.sample(frac=0.2, random_state=101))
train = temp.drop(validation.index)

C:\Users\austg\AppData\Local\Temp\ipykernel_13524\2877993709.py:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  validation = temp.groupby('_original_label', group_keys=False).apply(lambda x: x.sample(frac=0.2, random_state=101))


In [4]:
train

,male,female,_original_label,test
1,পাপন শালা রে রিমান্ডে নেওয়া দরকার,নাহার শালী রে রিমান্ডে নেওয়া দরকার,1,0
2,জিল্লুর রহমান স্যারের ছেলে এতো বড় জারজ হবে এটা...,নূর জাহান ম্যাডামের মেয়ে এতো বড় জারজ হবে এটা ...,1,0
4,তুই তো শালা গাজা খাইছচ।তুর মার হেডায় খেলবে সাকিব,তুই তো শালী গাজা খাইছচ । তুর মার হেডায় খেলবে স...,1,0
6,পাপনে পাগল হয়াছে,দুর্গা পাগলিনী হয়াছে,1,0
9,ক্রিকেট মনে হয় ওনার বাবার সম্পত্তি হালার ফু,ক্রিকেট মনে হয় ওনার মায়ের সম্পত্তি হালীর ফু,1,0
...,...,...,...,...
9996,কুলাঙ্গার সন্তান তুই ধ্বংস হয়ে যাবি তোর বাবার...,কুলাঙ্গার সন্ত ান তুই ধ্বংস হয়ে যাবি তোর মায়...,0,0
9997,ভাই এরকম ভিডিও আরো করেন,আফা এরকম ভিডিও আরো করেন,0,0
9999,ভাই এরকম ভিডিও আরো করেন,ভাবী এরকম ভিডিও আরো করেন,0,0
10000,মূর্খ খালেদার ফাঁসি চাই,মূর্খা দুর্গা ফাঁসি চাই,0,0


In [5]:
validation

,male,female,_original_label,test
8406,এই মহিলার স্বামীর এখন কেমন প্রতিক্রিয়া সেটা এক...,এই পুরুষের স্ত্রীর এখন কেমন প্রতিক্রিয়া সেটা এ...,0,0
6001,ভাই সকিব বলে ছিল মাঠে নামার আগে সিদ্ধান্ত হয়ে ...,বোন সরলা বলে ছিল মাঠে নামার আগে সিদ্ধান্ত হয়...,0,0
5244,ভাই খারাপ লাগাটা অনেক বেশি কাজ করতেছে সত্যি হ্...,বোন খারাপ লাগাটা অনেক বেশি কাজ করতেছে সত্যি হ্...,0,0
8332,এত গুলো লোক দেখে তামাশা করল,এত গুলো স্ত্রীলোক দেখে তামাশা করল,0,0
7978,ভাই প্রশাসনের প্রতি আহ্বান জানান যেন মিননি কে ...,বোন প্রশাসনের প্রতি আহ্বান জানান যেন মিননি কে ...,0,0
...,...,...,...,...
1841,সাব্বাশ খানকির ছেলে,সাব্বাশ খানকির মেয়ে,1,0
4765,শুয়োর সালারে পিটিয়ে মেরে ফেলা হোক,শুয়োরনী সালী রে পিটিয়ে মেরে ফেলা হোক,1,0
4061,নূর খানকির পোলা জামাতের দালাল,নূর খানকির মাইয়া জামাতের দালাল,1,0
3692,হিন্দু পুরুষ বাংলাদেশে কেন,হিন্দু মহিলা বাংলাদেশে কেন,1,0


In [6]:
test

,male,female,_original_label,test
3,শালা লুচ্চা দেখতে পাঠার মত দেখা যায়,শালী লুচ্চা দেখতে পাঠীর মত দেখা যায়,1,1
8,দেখেছো মাইগ্যা না হিজরা বোঝা যাচ্ছে না । দেখে...,দেখেছো মাগি না হিজরা বোঝা যাচ্ছে না। দেখে মনে ...,1,1
10,সব মানুষ আপন হয় না জেমন পাপন আপন হইতে পারে না ...,সব মানুষ আপন হয় না জেমন রাইসা আপন হইতে পারে না...,1,1
17,এই চুদির ভাই তুই কোথায় শুনলি সাকিব ইংল্যান্ডেে...,এই চুদির আফা তুই কোথায় শুনলি সাকিব ইংল্যান্ডেে...,1,1
20,পাপন সালা পাগল হয়ে গেছে সবাই মিলে একে জুতা মারো,সাহানা সালী পাগলী হয়ে গেছে সবাই মিলে একে জুতা...,1,1
...,...,...,...,...
9961,স্বামী স্ত্রী কে ভালবেসে মিষ্টি খাওয়াইছে বলেএই...,স্ত্রী স্বামী কে ভালবেসে মিষ্টি খাওয়াইছে বলেএই...,0,1
9966,রানু পাগলি কে খমা করা উচিত নয়,আহনাফ পাগলা কে খমা করা উচিত নয়,0,1
9969,তাহসানের উচিৎ ভালো ছেলে দেখে বিয়ে করা বাজে একট...,তাহসানের উচিৎ ভালো মেয়ে দেখে বিয়ে করা বাজে একট...,0,1
9987,শাবনুর মহা নায়ক,শাবনুর মহা নায়িকা,0,1


# Minimizing KL Divergence

## Load Pretrained Model

In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transform
from torch.utils.data import DataLoader
from transformers import AutoTokenizer, AutoModelForMaskedLM, BertConfig
import pandas as pd
import numpy as np
import transformers
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch.optim as optim
import torch.nn.functional as F
from torchsummary import summary
from tqdm import tqdm
from tqdm import tqdm
import torch
import torch.optim as optim
import torch.nn.functional as F
from collections import OrderedDict
from datetime import datetime
from transformers import get_linear_schedule_with_warmup

def load_transformer_based_model(model_path):
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    model = AutoModelForMaskedLM.from_pretrained(model_path ,output_hidden_states=True)
    return tokenizer, model
model_path = "Hate-speech-CNERG/bengali-abusive-MuRIL"
tokenizer, b_model = load_transformer_based_model(model_path)
print('Total Pretrained tokens: ',len(tokenizer.get_vocab()))
tokenizer.add_tokens(['<Name>' ,'<Gender>'])
b_model.resize_token_embeddings(len(tokenizer))
print('Total number of tokens after adding the new ones: ',len(tokenizer.get_vocab()))

c:\Users\austg\.conda\envs\Temp\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\austg\.conda\envs\Temp\Lib\site-packages\huggingface_hub\file_download.py:144: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\austg\.cache\huggingface\hub\models--Hate-speech-CNERG--bengali-abusive-MuRIL. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to act

Total Pretrained tokens:  197285


The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


Total number of tokens after adding the new ones:  197287


## Creating Custom Data Loader

In [8]:
class BertDataset(Dataset):
    def __init__(self, data, tokenizer,max_length):
        super(BertDataset, self).__init__()

        self.train_csv= data
        self.tokenizer=tokenizer
        self.target=self.train_csv.iloc[:,2]
        self.max_length=max_length

    def __len__(self):
        return len(self.train_csv)

    def __getitem__(self, index):

        text1 = self.train_csv.iloc[index,0] #0th column male text
        text2 = self.train_csv.iloc[index,1] #1st column female text

        inputs = self.tokenizer.batch_encode_plus(
            [text1, text2] ,
            pad_to_max_length=True,
            add_special_tokens=True,
            return_attention_mask=True,
            max_length=self.max_length,
            truncation = True
        )
        ids = inputs["input_ids"]
        token_type_ids = inputs["token_type_ids"]
        masks = inputs["attention_mask"]

        return torch.tensor(ids[0], dtype=torch.long), torch.tensor(token_type_ids[0], dtype=torch.long), torch.tensor(masks[0], dtype=torch.long), torch.tensor(ids[1], dtype=torch.long), torch.tensor(token_type_ids[1], dtype=torch.long), torch.tensor(masks[1], dtype=torch.long), torch.tensor(self.train_csv.iloc[index, 2], dtype=torch.long)

BATCH_SIZE = 4

dataset_train= BertDataset(train, tokenizer, max_length=512 )
dataset_val= BertDataset( validation, tokenizer, max_length=512)
dataset_test= BertDataset(test, tokenizer, max_length=512)
train_loader=DataLoader(dataset=dataset_train,batch_size=BATCH_SIZE, shuffle=True)
validation_loader=DataLoader(dataset=dataset_val,batch_size=BATCH_SIZE, shuffle=False)
test_loader=DataLoader(dataset=dataset_test,batch_size=BATCH_SIZE, shuffle=False)

## Creating Custom Model

In [9]:
import numpy as np
np.random.seed()
class BERT(nn.Module):
    def __init__(self, model):
        super(BERT, self).__init__()
        self.bert_model = model
        # print(self.bert_model.bert.embeddings.word_embeddings.weight)
        self.dropout = nn.Dropout(0.2)
        self.out = nn.Linear(768, 2)

    def forward(self,ids1, mask1, token_type_ids1, ids2, mask2, token_type_ids2, mode):
        if mode == 'training':
          ids = torch.vstack((ids1, ids2))
          mask = torch.vstack((mask1, mask2))
          token_type_ids = torch.vstack((token_type_ids1, token_type_ids2))
          o2= self.bert_model(ids,mask,token_type_ids).hidden_states[0]
          o2= o2.max(dim=1).values
          male_embedding, female_embedding = torch.vsplit(o2, 2)
          # print(original_embedding.shape, ner_embedding.shape)
          o2_male = self.dropout(male_embedding)
          out_male= self.out(o2_male)

          o2_female = self.dropout(female_embedding)
          out_female= self.out(o2_female)


          #randomization
          c = np.random.randint(0, 2)
          output = out_male if c==0 else out_female

          return output, F.softmax(out_male, dim=-1), F.softmax(out_female, dim=-1)


        if mode == 'inference':
          o2= self.bert_model(ids1, mask1, token_type_ids1).hidden_states[0]
          oe= o2.max(dim=1).values
          o2 = self.dropout(oe)
          out= self.out(o2)
          return out, oe

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
masked_model = nn.DataParallel(BERT(b_model)).to(device)

In [10]:
#debug
i1, t1,m1,i2,t2,m2,l = next(iter(train_loader))
i1 = i1.to(device)
t1 = t1.to(device)
m1 = m1.to(device)
i2 = i2.to(device)
t2 = t2.to(device)
m2 = m2.to(device)
l = l.to(device)
print('Input Shape: ', i1.shape)

logits, male_softmax, female_softmax = masked_model(i1, t1,m1,i2,t2,m2, 'training')
print(logits.shape, male_softmax.shape, male_softmax.shape)
logits, embedding = masked_model(i1, t1,m1,None,None,None, 'inference')
print(logits.shape, embedding.shape)

c:\Users\austg\.conda\envs\Temp\Lib\site-packages\transformers\tokenization_utils_base.py:2700: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Input Shape:  torch.Size([4, 512])
torch.Size([4, 2]) torch.Size([4, 2]) torch.Size([4, 2])
torch.Size([4, 2]) torch.Size([4, 768])


In [ ]:
logits, logits.max(1)[1].view(-1,1)

In [ ]:
logits.max(1)[1].view(-1,1).eq(l.view(-1,1)).sum().item()

## Creating Utility Class for Tracking Training Stat

In [11]:
from collections import OrderedDict
from collections import namedtuple
from itertools import product
import json
from torch.utils.tensorboard import SummaryWriter
import time
import torchvision
import pandas as pd
import torch
from IPython.display import display
from IPython.display import clear_output

class RunBuilder():
    @staticmethod
    def get_runs(params):
        Run = namedtuple('Run', params.keys())


        runs = []
        for v in product(*params.values()):
            runs.append(Run(*v))


        return runs


class RunManager():
    def __init__(self):
        self.epoch_count = 0
        self.epoch_ce_loss = 0
        self.epoch_kl_loss = 0
        self.epoch_loss = 0
        self.epoch_num_correct = 0
        self.epoch_start_time = None

        self.epoch_valid_ce_loss = 0.0
        self.epoch_valid_kl_loss = 0.0
        self.epoch_valid_loss = 0.0
        self.epoch_num_valid_correct = 0

        self.run_params = None
        self.run_count = 0
        self.run_data = []
        self.run_start_time = None

        self.network = None
        self.loader = None
        self.validation_loader = None
        self.tb = None

    def begin_run(self, run, network, loader, validation_loader):
        self.run_start_time = time.time()

        self.run_params = run
        self.run_count += 1

        self.network = network
        self.loader = loader
        self.validation_loader = validation_loader
        self.tb = SummaryWriter(comment=f'-{run}')



    def end_run(self):
        self.tb.close()
        self.epoch_count = 0

    def begin_epoch(self):
        self.epoch_start_time = time.time()
        self.epoch_count += 1
        self.epoch_loss = 0
        self.epoch_ce_loss = 0
        self.epoch_kl_loss = 0
        self.epoch_num_correct = 0
        self.epoch_valid_ce_loss = 0
        self.epoch_valid_kl_loss = 0
        self.epoch_valid_loss = 0
        self.epoch_num_valid_correct = 0

    def end_epoch(self):
        epoch_duration = time.time() - self.epoch_start_time
        run_duration = time.time() - self.run_start_time

        ce_loss = self.epoch_ce_loss/len(self.loader.dataset)
        kl_loss = self.epoch_kl_loss/len(self.loader.dataset)
        loss = self.epoch_loss/len(self.loader.dataset)
        accuracy = self.epoch_num_correct/len(self.loader.dataset)
        val_ce_loss = self.epoch_valid_ce_loss/len(self.validation_loader.dataset)
        val_kl_loss = self.epoch_valid_kl_loss/len(self.validation_loader.dataset)
        val_loss = self.epoch_valid_loss/len(self.validation_loader.dataset)
        val_accuracy = self.epoch_num_valid_correct/len(self.validation_loader.dataset)

        self.tb.add_scalar('CE Loss', ce_loss, self.epoch_count)
        self.tb.add_scalar('KL Loss', kl_loss, self.epoch_count)
        self.tb.add_scalar('Loss', loss, self.epoch_count)
        self.tb.add_scalar('Accuracy', accuracy, self.epoch_count)
        self.tb.add_scalar('Validation CE Loss', val_ce_loss, self.epoch_count)
        self.tb.add_scalar('Validation KL Loss', val_kl_loss, self.epoch_count)
        self.tb.add_scalar('Validation Loss', val_loss, self.epoch_count)
        self.tb.add_scalar('Validation Accuracy', val_accuracy, self.epoch_count)
        print('-------------')
        for name, param in self.network.named_parameters():
            # print(name, param, self.epoch_count)
            self.tb.add_histogram(name, param, self.epoch_count)

        results = OrderedDict()
        results["run"] = self.run_count
        results["epoch"] = self.epoch_count
        results["ce_loss"] = ce_loss
        results["kl_loss"] = kl_loss
        results["loss"] = loss
        results["accuracy"] = accuracy
        results["validation ce loss"] = val_ce_loss
        results["validation kl loss"] = val_kl_loss
        results["validation loss"] = val_loss
        results["validation accuracy"] = val_accuracy
        results["epoch duration (minutes)"] = epoch_duration/60
        results["run duration (minutes)"] = run_duration/60
        for k,v in self.run_params._asdict().items(): results[k] = v
        self.run_data.append(results)
        df = pd.DataFrame.from_dict(self.run_data, orient='columns')

        clear_output(wait=True)
        display(df)

    def track_loss(self, ce_loss, kl_loss, loss):
        self.epoch_ce_loss += ce_loss.item()#*self.loader.batch_size
        self.epoch_kl_loss += kl_loss.item()#*self.loader.batch_size
        self.epoch_loss += loss.item()#*self.loader.batch_size

    def track_num_correct(self, preds, labels):
        self.epoch_num_correct += self._get_num_correct(preds, labels)

    def track_validation_loss(self, ce_loss, kl_loss, loss):
        self.epoch_valid_ce_loss += ce_loss.item()#*self.loader.batch_size
        self.epoch_valid_kl_loss += kl_loss.item()#*self.loader.batch_size
        self.epoch_valid_loss += loss.item()#*self.loader.batch_size

    def track_num_validation_correct(self, preds, labels):
        self.epoch_num_valid_correct += self._get_num_correct(preds, labels)

    @torch.no_grad()
    def _get_num_correct(self, preds, labels):
        return preds.max(1)[1].view(-1,1).eq(labels).sum().item()

    def save(self, fileName):
        pd.DataFrame.from_dict(
            self.run_data,
            orient='columns').to_csv(rf'{fileName}.csv')

## Creating Training Loop

In [12]:
SAVE_MODEL = True
def my_trainer(train_loader, validation_loader, network, LR = 1e-5 ,epochs = 5, BATCH_SIZE = 16,  fresh_training = True, validation = True):
    params = OrderedDict(
        lr = [LR],
        batch_size =[BATCH_SIZE],
        device = ['cuda' if torch.cuda.is_available() else 'cpu'],
    )

    m = RunManager()
    for run in RunBuilder.get_runs(params):
        device = torch.device(run.device)
        # network = network.to(device)
        # optimizer = optim.Adam(network.parameters(), lr=run.lr)

        loss_fn = nn.CrossEntropyLoss()
        loss_fn_2 = nn.KLDivLoss(reduction="batchmean")

        total_steps = len(train_loader) * epochs
        optimizer = optim.Adam(network.parameters(), lr= run.lr)

        scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps = 0, num_training_steps = total_steps)
        if not fresh_training:
            load_model(network, optimizer, PATH = 'HateSpeech_KLDM_Randomized.pth')
        m.begin_run(run, network, train_loader, validation_loader)


        for epoch in range(epochs):
            network.train(True)
            print(f'Epoch {epoch + 1} : ', end = '\t')
            m.begin_epoch()


            for batch in tqdm(train_loader):
                input_ids1, token_type_ids1, attention_masks1, input_ids2, token_type_ids2, attention_masks2, labels = batch
                input_ids1 = input_ids1.to(device)
                token_type_ids1 = token_type_ids1.to(device)
                attention_masks1 = attention_masks1.to(device)

                input_ids2 = input_ids2.to(device)
                token_type_ids2 = token_type_ids2.to(device)
                attention_masks2 = attention_masks2.to(device)

                labels = labels.type(torch.LongTensor)
                labels = labels.to(device)
                copy_labels = labels

                preds, male_softmax, female_softmax = network(input_ids1, attention_masks1, token_type_ids1, input_ids2, attention_masks2, token_type_ids2, 'training')
                labels = labels.view(-1, 1)



                loss1 = loss_fn(preds, copy_labels)
                loss2 = loss_fn_2(male_softmax.log(), female_softmax) + loss_fn_2(female_softmax.log(), male_softmax)
                loss = loss1 + loss2
                # print(preds.shape, labels.shape)
                # print(preds, labels)


                optimizer.zero_grad()
                loss.backward()

                torch.nn.utils.clip_grad_norm_(network.parameters(), 1.0)

                optimizer.step()

                scheduler.step()


                m.track_loss(loss1, loss2, loss)
                m.track_num_correct(preds, labels)


            network.eval()
            # print(network.bert_model.bert.embeddings.word_embeddings.weight)
            for batch in tqdm(validation_loader):
                input_ids1, token_type_ids1, attention_masks1, input_ids2, token_type_ids2, attention_masks2, labels = batch
                input_ids1 = input_ids1.to(device)
                token_type_ids1 = token_type_ids1.to(device)
                attention_masks1 = attention_masks1.to(device)
                input_ids2 = input_ids2.to(device)
                token_type_ids2 = token_type_ids2.to(device)
                attention_masks2 = attention_masks2.to(device)

                labels = labels.type(torch.LongTensor)
                labels = labels.to(device)
                copy_labels = labels
                with torch.no_grad():
                    preds, male_softmax, female_softmax = network(input_ids1, attention_masks1, token_type_ids1, input_ids2, attention_masks2, token_type_ids2, 'training')
                    labels = labels.view(-1, 1)
                    loss1 = loss_fn(preds, copy_labels)
                    loss2 = loss_fn_2(male_softmax.log(), female_softmax) + loss_fn_2(female_softmax.log(),male_softmax)
                    loss = loss1 + loss2
                    m.track_validation_loss(loss1, loss2, loss)
                    m.track_num_validation_correct(preds, labels)
                # break #not doing validation here




            m.end_epoch()
            network.train(False)
            save_model(network, optimizer)
        m.end_run()
    if SAVE_MODEL == True:
        save_model(network, optimizer)

    s = datetime.now().strftime("%d-%m-%Y %I:%M:%p").replace(" ", "_").replace(":", "_")
    PATH  = rf'results_{s}'
    m.save(PATH)
    return


def save_model(network, optimizer):
    s = datetime.now().strftime("%d-%m-%Y %I:%M:%p").replace(" ", "_").replace(":", "_")
    PATH  = rf'HateSpeech_KLDM_Randomized.pth'
    torch.save({
    'model_state_dict': network.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    }, PATH)
    print(f'Saved model and optimizer at {PATH}')
    return

def load_model(network, optimizer, PATH ):
    checkpoint = torch.load(PATH)
    network.load_state_dict(checkpoint['model_state_dict'])
    if optimizer is not None:
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        print(rf'Loaded model and optimizer from {PATH}')
    else:
        print(rf'Loaded model from {PATH}')

    return

def get_parameter_info(network, details = False):
    print('Total parameters:' ,(sum(p.numel() for p in network.parameters())))
    print('Total trainable parameters:' ,(sum(p.numel() for p in network.parameters() if p.requires_grad)))
    if details:
        print('---------------------------------\nDetailed Parameter Info: ')

        for name, parameter in network.named_parameters():
            print('name, parameter.requires_grad, parameter.is_cuda, parameter.size(): ', name, parameter.requires_grad, parameter.is_cuda, parameter.size())
            print('---------------------------------')
    return

if finetuning:
  my_trainer(train_loader, validation_loader, masked_model, 0.0001, epochs = 15, BATCH_SIZE = BATCH_SIZE, fresh_training = True)

,run,epoch,ce_loss,kl_loss,loss,accuracy,validation ce loss,validation kl loss,validation loss,validation accuracy,epoch duration (minutes),run duration (minutes),lr,batch_size,device
0,1,1,0.116611,0.010697,0.127309,0.778351,0.077200,0.005024,0.082224,0.873125,2.345351,2.345384,0.0001,4,cuda
1,1,2,0.065476,0.011091,0.076567,0.898938,0.069334,0.003525,0.072859,0.895000,2.347591,4.905676,0.0001,4,cuda
2,1,3,0.053115,0.010210,0.063325,0.922212,0.072591,0.004341,0.076932,0.899375,2.356197,7.476417,0.0001,4,cuda
3,1,4,0.043217,0.008671,0.051887,0.941112,0.065640,0.004687,0.070327,0.914375,2.357955,10.047495,0.0001,4,cuda
4,1,5,0.037145,0.008860,0.046005,0.952202,0.067936,0.005812,0.073748,0.915000,2.359524,12.621681,0.0001,4,cuda
5,1,6,0.031186,0.008280,0.039465,0.960637,0.068118,0.006178,0.074296,0.915625,2.347795,15.181389,0.0001,4,cuda
6,1,7,0.025924,0.007255,0.033178,0.968447,0.069172,0.006196,0.075368,0.918125,2.353467,17.747838,0.0001,4,cuda
7,1,8,0.022288,0.007554,0.029842,0.973602,0.074488,0.006754,0.081242,0.917500,2.351797,20.312515,0.0001,4,cuda
8,1,9,0.019143,0.005998,0.025140,0.977976,0.069868,0.007324,0.077193,0.920625,2.354596,22.879703,0.0001,4,cuda
9,1,10,0.016539,0.005715,0.022254,0.981412,0.074172,0.007450,0.081622,0.916875,2.349453,25.444139,0.0001,4,cuda


Saved model and optimizer at HateSpeech_KLDM_Randomized.pth
Saved model and optimizer at HateSpeech_KLDM_Randomized.pth


## Load Saved Model

In [13]:
#Load Model
from tqdm import tqdm
import torch
import torch.optim as optim
import torch.nn.functional as F
from collections import OrderedDict
from datetime import datetime
from transformers import get_linear_schedule_with_warmup

def load_model(network, optimizer, PATH ):
    checkpoint = torch.load(PATH, map_location=torch.device('cpu'))
    network.load_state_dict(checkpoint['model_state_dict'])
    if optimizer is not None:
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        print(rf'Loaded model and optimizer from {PATH}')
    else:
        print(rf'Loaded model from {PATH}')

    return


if load_and_run:
#   load_model(masked_model, None, PATH = '/kaggle/input/toxicity_kldm/pytorch/default/1/Toxicity_KLDM.pth')
  load_model(masked_model, None, PATH = 'HateSpeech_KLDM_Randomized.pth')

## Inference

In [14]:
import random
import warnings
warnings.filterwarnings('ignore')
def get_prediction(data_loader, network, choice = 'original'):
  device = 'cuda' if torch.cuda.is_available() else 'cpu'
  device = torch.device(device)
  network = network.to(device)
  network.eval()
  l = []
  emb = []
  for batch in tqdm(data_loader):
    input_ids1, token_type_ids1, attention_masks1, input_ids2, token_type_ids2, attention_masks2, labels = batch
    input_ids1 = input_ids1.to(device)
    token_type_ids1 = token_type_ids1.to(device)
    attention_masks1 = attention_masks1.to(device)
    input_ids2 = input_ids2.to(device)
    token_type_ids2 = token_type_ids2.to(device)
    attention_masks2 = attention_masks2.to(device)
    labels = labels.type(torch.LongTensor)
    labels = labels.to(device)
    with torch.no_grad():
        if choice == 'male':
          preds, hidden_states = network(input_ids1, attention_masks1,token_type_ids1, None, None, None, 'inference')
        else:
          preds, hidden_states = network(input_ids2, attention_masks2,token_type_ids2, None, None, None, 'inference')

        labels = labels.type_as(preds).view(-1, 1)
        preds =  preds.max(1)[1].view(-1,1)
        l.append(preds)
        emb.append(hidden_states)
  return l, torch.vstack(emb).cpu().numpy()

p, emb_male = get_prediction(test_loader, masked_model, 'male')
test['pred_male_kldm'] = [x[0] for pred in p for x in pred.tolist()]
p, emb_male = get_prediction(test_loader, masked_model, 'female')
test['pred_female_kldm'] = [x[0] for pred in p for x in pred.tolist()]
x = sum(test['pred_male_kldm']!=test['_original_label'])
print('For Male:\nTotal test data: ', len(test),'Total mismatch: ', x,'Accuracy: ', 1- (x/len(test))) 
x = sum(test['pred_female_kldm']!=test['_original_label'])
print('For Female:\nTotal test data: ', len(test),'Total mismatch: ', x,'Accuracy: ', 1- (x/len(test)))
print('Male Female Mismatch: ', sum(test['pred_male_kldm']!=test['pred_female_kldm']))

100%|██████████| 500/500 [00:14<00:00, 35.39it/s]

For Male:
Total test data:  2000 Total mismatch:  231 Accuracy:  0.8845
For Female:
Total test data:  2000 Total mismatch:  227 Accuracy:  0.8865
Male Female Mismatch:  46


# KLD Minimization Result

In [15]:
import numpy as np
np.random.seed(0)

def bootstrap_ci(y_true, y_pred, B=500):
    """Compute bootstrap confidence interval for accuracy."""
    N = len(y_true)
    original_acc = np.mean(y_pred == y_true)

    bootstrap_accs = []

    for _ in range(B):
        indices = np.random.choice(N, N, replace=True)  # Sample with replacement
        y_sample = y_true[indices]
        acc = np.mean(y_pred[indices] == y_sample)
        bootstrap_accs.append(acc)
    # Compute 95% confidence interval
    lower, upper = np.percentile(bootstrap_accs, [2.5, 97.5])

    return original_acc, (lower, upper), np.mean(bootstrap_accs)

y_true = test['_original_label'].values  # Ground truth labels
y_pred = test['pred_male_kldm'].values
acc, ci, ma = bootstrap_ci(y_true, y_pred)
print('Male: ')
print(f"Accuracy: {acc*100:.2f}")
print(f"95% Confidence Interval: {ci[0]*100:.2f}  {ci[1]*100:.2f}")
print(f"mean accuracy: {ma*100: .2f}")

y_true = test['_original_label'].values
y_pred = test['pred_female_kldm'].values
acc, ci, ma = bootstrap_ci(y_true, y_pred)
print('\nFemale: ')
print(f"Accuracy: {acc*100:.2f}")
print(f"95% Confidence Interval: {ci[0]*100:.2f}  {ci[1]*100:.2f}")
print(f"mean accuracy: {ma*100: .2f}")
print('Male Female Mismatch: ', sum(test['pred_male_kldm']!=test['pred_female_kldm']))
test.to_csv('revision_kldm_randomized_hatespeech_result.csv', index = False)
print('Saved....')

Male: 
Accuracy: 88.45
95% Confidence Interval: 86.97  89.80
mean accuracy:  88.39

Female: 
Accuracy: 88.65
95% Confidence Interval: 87.35  90.05
mean accuracy:  88.66
Male Female Mismatch:  46
Saved....


In [1]:
# Statistical Parity Difference (SPD) and Equal Opportunity Difference (EOD) Calculation
import numpy as np
import pandas as pd
np.random.seed(0)

test = pd.read_csv('revision_kldm_randomized_hatespeech_result.csv')


# Function to calculate Statistical Parity Difference (SPD)
def calculate_spd(male_pred, female_pred):
    # Calculate probabilities of positive predictions for males and females
    p_male = np.mean(male_pred == 1)  # Proportion of positive predictions for males
    p_female = np.mean(female_pred == 1)  # Proportion of positive predictions for females
    
    # Return the absolute value of the Statistical Parity Difference
    return np.abs(p_male - p_female)

# Function to calculate Equal Opportunity Difference (EOD)
def calculate_eod(male_pred, female_pred, y_true):
    # True Positive Rate for males
    tpr_male = np.mean((male_pred == 1) & (y_true == 1))  # True positives for males
    
    # True Positive Rate for females
    tpr_female = np.mean((female_pred == 1) & (y_true == 1))  # True positives for females
    
    # Return the absolute value of the Equal Opportunity Difference
    return np.abs(tpr_male - tpr_female)

# Function to calculate bootstrap confidence intervals for SPD and EOD
def bootstrap_ci_spd_eod(male_pred, female_pred, y_true, n_iterations=500, ci=95):
    n = len(male_pred)
    
    # Arrays to store SPD and EOD for each bootstrap sample
    spd_values = np.zeros(n_iterations)
    eod_values = np.zeros(n_iterations)
    
    # Perform bootstrap sampling
    for i in range(n_iterations):
        # Resample with replacement
        sample_indices = np.random.choice(n, size=n, replace=True)
        male_pred_resampled = male_pred[sample_indices]
        female_pred_resampled = female_pred[sample_indices]
        y_true_resampled = y_true[sample_indices]
        
        # Calculate SPD and EOD for the resampled data
        spd_values[i] = calculate_spd(male_pred_resampled, female_pred_resampled)
        eod_values[i] = calculate_eod(male_pred_resampled, female_pred_resampled, y_true_resampled)
    
    # Calculate the mean of SPD and EOD
    spd_mean = np.mean(spd_values)
    eod_mean = np.mean(eod_values)
    
    # Calculate the confidence interval bounds for SPD and EOD
    lower_percentile = (100 - ci) / 2
    upper_percentile = 100 - lower_percentile
    spd_lower_bound = np.percentile(spd_values, lower_percentile)
    spd_upper_bound = np.percentile(spd_values, upper_percentile)
    eod_lower_bound = np.percentile(eod_values, lower_percentile)
    eod_upper_bound = np.percentile(eod_values, upper_percentile)
    
    return (np.abs(spd_mean), np.abs(spd_lower_bound), np.abs(spd_upper_bound)), (np.abs(eod_mean), np.abs(eod_lower_bound), np.abs(eod_upper_bound))


y_true = test._original_label.values
male_pred = test.pred_male_kldm.values
female_pred = test.pred_female_kldm.values

# Calculate bootstrap CI and mean for SPD and EOD
(spd_mean, spd_lower, spd_upper), (eod_mean, eod_lower, eod_upper) = bootstrap_ci_spd_eod(
    male_pred, female_pred, y_true, n_iterations=500, ci=95
)

print(f"SPD 95% Confidence Interval: [{spd_lower:.3f}, {spd_upper:.3f}]\tSPD Mean: {spd_mean:.3f}")
print('---------------------------------------------------------------------------------------------')
print(f"EOD 95% Confidence Interval: [{eod_lower:.3f}, {eod_upper:.3f}]\tEOD Mean: {eod_mean:.3f}")

c:\Users\joyau\anaconda3\envs\pytorch_gpu\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


SPD 95% Confidence Interval: [0.000, 0.008]	SPD Mean: 0.003
---------------------------------------------------------------------------------------------
EOD 95% Confidence Interval: [0.000, 0.005]	EOD Mean: 0.002


In [ ]:
# Male: 
# Accuracy: 0.9043
# 95% Confidence Interval: (np.float64(0.8882801418439716), np.float64(0.9198936170212766))
# mean accuracy:  90.39

# Female: 
# Accuracy: 0.8950
# 95% Confidence Interval: (np.float64(0.8797695035460993), np.float64(0.9113475177304965))
# mean accuracy:  89.50
# Male Female Mismatch:  33
# Saved....

In [ ]:
# Male: 
# Accuracy: 0.9071
# 95% Confidence Interval: (0.8918262411347517, 0.9223581560283688)
# mean accuracy: 0.9068921985815602

# Female: 
# Accuracy: 0.8965
# 95% Confidence Interval: (0.8818971631205674, 0.9113475177304965)
# mean accuracy: 0.896463829787234
# Male Female Mismatch:  43
# Saved....

## Result of Approach 2

In [ ]:
x = sum(test['original_lebel'] != test['a2_original_pred'])
print('For Original text:\nTotal test data: ', len(test),'Total mismatch: ', x,'Accuracy: ', 1 - (x/len(test)) )
x = sum(test['original_lebel'] != test['a2_ner_pred'])
print('For NER Converted text:\nTotal test data: ', len(test),'Total mismatch: ', x,'Accuracy: ', 1 - (x/len(test)))
print('Bias: ', sum(test['a2_ner_pred'] != test['a2_original_pred']))

In [ ]:
x = sum(test['original_lebel'] != test['a2_original_pred'])
print('For Original text:\nTotal test data: ', len(test),'Total mismatch: ', x,'Accuracy: ', 1 - (x/len(test)) )
x = sum(test['original_lebel'] != test['a2_ner_pred'])
print('For NER Converted text:\nTotal test data: ', len(test),'Total mismatch: ', x,'Accuracy: ', 1 - (x/len(test)))
print('Bias: ', sum(test['a2_ner_pred'] != test['a2_original_pred']))

In [ ]:
x = test[test['a2_ner_pred'] != test['a2_original_pred']][['Original Sentence', 'Converted Sentence', 'original_lebel', 'a2_original_pred', 'a2_ner_pred']]
print(len(x))
x

## Sentence Embedding Visualization for Original Sentence and NER Converted Sentence

### For Training Data

In [ ]:
#for visualizing training data embedding
dataset_train= BertDataset(train, tokenizer, max_length=200)
train_loader=DataLoader(dataset=dataset_train,batch_size=16, shuffle = False)
p, emb_ori = get_prediction(train_loader, masked_model, 'original')
p, emb_ner = get_prediction(train_loader, masked_model, 'ner')

In [ ]:
from sklearn.manifold import TSNE
import plotly.express as px

tsne_cls = TSNE(n_components=2, random_state=42).fit_transform(emb_ori)

labels = train['original_lebel']   # Extract labels
review_texts = train['Original Sentence']  # Extract review texts

# Prepare the data for Plotly plots
plot_data_cls = {'t-SNE Dimension 1': tsne_cls[:, 0],
                 't-SNE Dimension 2': tsne_cls[:, 1],
                 'Label': labels,
                 'Text': review_texts}

# Customize colorbar to show integer labels
colorbar_tickvals = list(range(1, 6))
colorbar_ticktext = [str(val) for val in colorbar_tickvals]

# Plotting with Plotly - [CLS] Token Embeddings
fig_cls = px.scatter(plot_data_cls, x='t-SNE Dimension 1', y='t-SNE Dimension 2',
                     color='Label', hover_data={'t-SNE Dimension 1': False, 't-SNE Dimension 2': False, 'Text': True})


fig_cls.update_layout(title='t-SNE of Embeddings Original Training Data with Labels', height=700,  font=dict(
        size=20,
))
fig_cls.update_traces(marker=dict(size=10, opacity=0.9))

fig_cls.update_layout(coloraxis_colorbar=dict(
    tickvals=colorbar_tickvals,
    ticktext=colorbar_ticktext
))

fig_cls.show()


In [ ]:
from sklearn.manifold import TSNE
import plotly.express as px

tsne_cls = TSNE(n_components=2, random_state=42).fit_transform(emb_ner)

labels = train['original_lebel']   # Extract labels
review_texts = train['Converted Sentence']  # Extract review texts

# Prepare the data for Plotly plots
plot_data_cls = {'t-SNE Dimension 1': tsne_cls[:, 0],
                 't-SNE Dimension 2': tsne_cls[:, 1],
                 'Label': labels,
                 'Text': review_texts}

# Customize colorbar to show integer labels
colorbar_tickvals = list(range(1, 6))
colorbar_ticktext = [str(val) for val in colorbar_tickvals]

# Plotting with Plotly - [CLS] Token Embeddings
fig_cls = px.scatter(plot_data_cls, x='t-SNE Dimension 1', y='t-SNE Dimension 2',
                     color='Label', hover_data={'t-SNE Dimension 1': False, 't-SNE Dimension 2': False, 'Text': True})


fig_cls.update_layout(title='t-SNE of Embeddings NER Converted Training Data with Labels', height=700,  font=dict(
        size=20,
))
fig_cls.update_traces(marker=dict(size=10, opacity=0.9))

fig_cls.update_layout(coloraxis_colorbar=dict(
    tickvals=colorbar_tickvals,
    ticktext=colorbar_ticktext
))

fig_cls.show()


### For Biased Test Data

In [ ]:
#for visualizing training data embedding
dataset_train= BertDataset(x, tokenizer, max_length=200)
train_loader=DataLoader(dataset=dataset_train,batch_size=16, shuffle = False)
p, emb_ori = get_prediction(train_loader, masked_model, 'original')
p, emb_ner = get_prediction(train_loader, masked_model, 'ner')

In [ ]:
from sklearn.manifold import TSNE
import plotly.express as px

tsne_cls = TSNE(n_components=2, random_state=42).fit_transform(emb_ori)

labels = x['original_lebel']   # Extract labels
review_texts = x['Original Sentence']  # Extract review texts

# Prepare the data for Plotly plots
plot_data_cls = {'t-SNE Dimension 1': tsne_cls[:, 0],
                 't-SNE Dimension 2': tsne_cls[:, 1],
                 'Label': labels,
                 'Text': review_texts}

# Customize colorbar to show integer labels
colorbar_tickvals = list(range(1, 6))
colorbar_ticktext = [str(val) for val in colorbar_tickvals]

# Plotting with Plotly - [CLS] Token Embeddings
fig_cls = px.scatter(plot_data_cls, x='t-SNE Dimension 1', y='t-SNE Dimension 2',
                     color='Label', hover_data={'t-SNE Dimension 1': False, 't-SNE Dimension 2': False, 'Text': True})


fig_cls.update_layout(title='t-SNE of Embeddings Original Training Data with Labels', height=700,  font=dict(
        size=20,
))
fig_cls.update_traces(marker=dict(size=10, opacity=0.9))

fig_cls.update_layout(coloraxis_colorbar=dict(
    tickvals=colorbar_tickvals,
    ticktext=colorbar_ticktext
))

fig_cls.show()

In [ ]:
from sklearn.manifold import TSNE
import plotly.express as px

tsne_cls = TSNE(n_components=2, random_state=42).fit_transform(emb_ner)

labels = x['original_lebel']   # Extract labels
review_texts = x['Converted Sentence']  # Extract review texts

# Prepare the data for Plotly plots
plot_data_cls = {'t-SNE Dimension 1': tsne_cls[:, 0],
                 't-SNE Dimension 2': tsne_cls[:, 1],
                 'Label': labels,
                 'Text': review_texts}

# Customize colorbar to show integer labels
colorbar_tickvals = list(range(1, 6))
colorbar_ticktext = [str(val) for val in colorbar_tickvals]

# Plotting with Plotly - [CLS] Token Embeddings
fig_cls = px.scatter(plot_data_cls, x='t-SNE Dimension 1', y='t-SNE Dimension 2',
                     color='Label', hover_data={'t-SNE Dimension 1': False, 't-SNE Dimension 2': False, 'Text': True})


fig_cls.update_layout(title='t-SNE of Embeddings NER Converted Training Data with Labels', height=700,  font=dict(
        size=20,
))
fig_cls.update_traces(marker=dict(size=10, opacity=0.9))

fig_cls.update_layout(coloraxis_colorbar=dict(
    tickvals=colorbar_tickvals,
    ticktext=colorbar_ticktext
))

fig_cls.show()

In [ ]:
test.to_csv('/content/drive/MyDrive/#Research/# GB/toxicity_reSult_a2.csv', index = False)